# MINI PROJECT BY -- DEEPANSHU UPADHYAYA
# SECTION -- CST
# UNIVERSITY ROLL NO -- 2014383

# importing required moules







In [13]:
#loading modules
import numpy as np, pandas as pd, matplotlib.pyplot as plt, random


In [14]:
import string,re

## download nltk module data and google translation module


In [15]:
pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import nltk
nltk.download('omw-1.4')



[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [17]:

foo_nltk=__import__('nltk')
foo_nltk.download('stopwords')
foo_nltk.download('punkt')
foo_nltk.download('averaged_perceptron_tagger')
foo_nltk.download('tagsets')
foo_nltk.download('treebank')
foo_nltk.download('wordnet')
foo_nltk.download('twitter_samples')

del foo_nltk

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


## load data 

In [18]:
# load data loader 
from nltk.corpus import twitter_samples

# get some random index list 
random_indexs=np.random.randint(0,high=5000,size=5)

In [38]:

tweets_data=twitter_samples.strings(fileids='tweets.20150430-223406.json')
# total number of tweets 
print('Total number of smaples:',len(tweets_data))
# see random tweets (any five)
for index in np.random.randint(0,high=len(tweets_data),size=5):
  # print random tweet
  print('Tweet number:',index,':',tweets_data[index])

Total number of smaples: 20000
Tweet number: 3579 : RT @AamerAnwar: What is the purpose of Willie Rennie? Serious question #SNP #VoteSNP
Tweet number: 15545 : RT @BBCDouglasF: Sturgeon #QE2015: a lot of her 30mins on the back foot over #indyref2 + health questions. Lacked usual clarity of key mess…
Tweet number: 5305 : RT @SkyNews: THE INDEPENDENT FRONT PAGE: "Miliband: I will not do deal with the SNP to become PM" #skypapers http://t.co/im5LfHEUij
Tweet number: 9044 : RT @andrewducker: Can someone explain to me what Ed Miliband is thinking? http://t.co/Esup0BLBrP
Tweet number: 6255 : RT @LabourLeft: "Clare Gerada quits NHS England to be able to speak out on Tories' NHS privatisation" via Pulse Today: http://t.co/oRGDAftc…


## Functions 

### data cleaning functions 

In [43]:
# make remove - "# tag" and "@ tag" strings
def tagRemover(data:list,to_remove:list=['#[\S]*','@[\S]*','http[\S]*'],
               filler=''):
  for pattern in to_remove:
    data=[re.sub(pattern,filler,tweet) for tweet in data] 
  return data 

# function to get set of similes in data
def getSimiles(data:list,pattern:str='[a-zA-Z0-9]*',filler=''):
  # make set of all similes used in twitter data 
  similes=set()
  for tweet in data:
    for simile in re.sub(pattern,filler,tweet).split(): 
      similes.add(simile)
  return similes

# function to remove special characters
def specialCharaterRemover(data:list,specialInData:set,to_leave:dict,
                           decodeLeaves:dict,filler=' '):
  #revmove all characters except alphabets
  to_remove=string.printable.replace(string.ascii_letters,'')
  to_remove_trans=str.maketrans(to_remove,filler*len(to_remove))
  # join all special characters in data - get all special characters in twitter data  
  special_charters=''.join(specialInData)
  # leave all special in special characters
  for key,value in to_leave.items():
    special_charters=special_charters.replace(key,value)
  special_charters_trans=str.maketrans(special_charters,filler*len(special_charters))
  data=[tweet.translate(special_charters_trans) for tweet in data]
  # convert ':(' or ':)' to word "negative" or "postive" 
  for key,value in decodeLeaves.items():
    data=[tweet.replace(key,value) for tweet in data]
  data=[tweet.translate(to_remove_trans) for tweet in data]
  return data

# load nltk tokenizer 
from nltk import word_tokenize
# load stopwords 
from nltk.corpus import stopwords  
#removing stopwords and words having length less than three
def removeStopWords(data:list,stopwords_list:list=stopwords.words(),minWordLen=2):
  # get list of all stopwords (of all languages)
  data2return=list()
  # clean tweets (after tokenizing)
  for tweet in data:
    data2return.append([word for word in word_tokenize(tweet) if (word not in stopwords_list) and len(word)>minWordLen])
  return data2return
  

### pre-processing functions 

In [42]:
# load Stemmer and Lemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag

# function to stem and lemmatize word 
def wordCleaner(data:list):
  # make Stemmer
  stemmer_=SnowballStemmer('english')
  # make lemmatizer
  lemmatizer_=WordNetLemmatizer()
  cleanedData=list()
  
  # make clean - sentence by sentence 
  for sentence in data:
    # add pos tags (and clean word by word)
    # make list of cleaned words 
    cleanSentences=list()
    for token,tag in pos_tag(sentence):
      # if tag start with NN
      if tag.startswith('NN'): 
        tag='n' # make noun 
      # if tag start with VB
      elif tag.startswith('VB'): 
        tag='v'  # verb
      
      else: 
        tag='a'
      # apply Lemmatization 
      token=lemmatizer_.lemmatize(token.lower(),tag)
      # apply stemmer 
      token=stemmer_.stem(token)
      # apped words to clean sentences list 
      cleanSentences.append(token)
    # append cleaned stentences to cleaned data list 
    cleanedData.append(cleanSentences)

  return cleanedData

# function make list to generator 
def getGenerator(data:list):
  for sentence in data:
    for token in sentence:
      yield token 

# function to get modeled token ductionary like -> {'word':True,'python':True}
def getModeledTokens(data:list):
  # get sentence from data 
  for sentence in data:
    yield dict([(token,True) for token in sentence])

## view data 

In [22]:
# positive tweets 
positive_tweets=twitter_samples.strings('positive_tweets.json')
# total number of positive tweets
print('Total number of positive smaples:',len(positive_tweets))
# see random tweets
for index in random_indexs:
  print('Tweet number:',index,':',positive_tweets[index])

Total number of positive smaples: 5000
Tweet number: 327 : Gratefulness = Happiness :) Post three things you're grateful for today in the comments
Tweet number: 554 : @zara_arsalan thnks progrmr :-)
Tweet number: 4084 : @MarkBreech Not sure it would be good thing 4 my bottom daring 2 say 2 Miss B but Im gonna be so stubborn on mouth soaping ! #NotHavingit :p
Tweet number: 61 : @MacatangayApril  follow @jnlazts &amp; http://t.co/RCvcYYO0Iq follow u back :)
Tweet number: 3728 : @iamBugti_18 &lt;------ff__special
Acha banda hai ✌ tweets bhi achi krta hai
Handsome bhi hai 😂
Must follow :)


In [23]:
#negative tweets 
negative_tweets=twitter_samples.strings('negative_tweets.json')
# total number of positive tweets
print('Total number of negative smaples:',len(negative_tweets))
# see random tweets 
for index in random_indexs:
  print('Tweet number:',index,':',negative_tweets[index])

Total number of positive smaples: 5000
Tweet number: 327 : Such a stressful and upsetting day yesterday, the UK government sucks :(
Tweet number: 554 : It just ruins your #FridayFeeling when you know next day you have to wake up early in the morning....:(
Tweet number: 4084 : Wish that I can buy all Bangtan's merch :(
Tweet number: 61 : @Mess0019 Well I am sure your work day is over before mine :(
Tweet number: 3728 : @vinrana1986 hii vin plss rply my tweet :((


## clean data 

In [24]:
## make remove - "# tag" and "@ tag" strings
# apply to all positive strings 
cleaned_positive_tweets=tagRemover(positive_tweets)
# apply to all negative strings 
cleaned_negative_tweets=tagRemover(negative_tweets)

#Finding similes used in data

#make set of all similes used in positive twitter data 
positive_similes=getSimiles(cleaned_positive_tweets)
#total positive similes found 
print('Total positive similes found:',len(positive_similes))
# see all found positive similies 
print('Positive similes:',positive_similes)

# make set of all similes used in negative twitter data 
negative_similes=getSimiles(cleaned_negative_tweets)
# count total negative similes found 
print('Total negative similes found:',len(negative_similes))
# see all found negative similies 
print('Negative similes:',negative_similes)

# do hand picking for positive similes 
positive_similes_code={':)':' positive ',
                       ':D':' positive ',
                       ':p':' positive '}

# do hand picking for negative similes 
negative_similes_code={':(':' negative '}    

## clean tweet data - remove and decode smilies 
to_prevent_positive={':':'',')':''}
# clean positive data 
cleaned_positive_tweets=specialCharaterRemover(cleaned_positive_tweets,positive_similes,to_prevent_positive,
                                               positive_similes_code)
# clean negative data
to_prevent_negative={':':'','(':''}
cleaned_negative_tweets=specialCharaterRemover(cleaned_negative_tweets,negative_similes,to_prevent_negative,
                                               negative_similes_code)

## make tokenize data 
# positive tweets 
cleaned_positive_tweets=removeStopWords(cleaned_positive_tweets)
# negative tweets 
cleaned_negative_tweets=removeStopWords(cleaned_negative_tweets)

Total positive similes found: 496
Positive similes: {':))', '¦', ':-)', '^-^', '((', '.........', '?-', '“”', '🇫🇷', '":)', '!!..;', '^^', '🐶', '♥♥', '«»', '★★★★', '(', '.:-)', '💜', '((-:', '/-\\', ':(', ':):)', '´', '_', '☺', ':-).', '...!!!', '||', "'.", '💓', '-!!', ',', '?:)', ':))😌', "'!!", '😤😤', '✧｡', '→', '-_-', '💞😘', '..!!!', ':)():)!:!;', '😂😁', '🔴', ':)💞', ']', '"-"', '-', '|', ':)???', '=', '-_', '"-', '👊🏻“', '../', '😂😂', '?(', '.!', '(?)', '😉', '🙈', '`', '__', '?!?!?!', '&;:&;', '★', '˚', '!!!!:=.', '!:)', '/??""', '=^.^=', '*', '£.', '))', '-?', '⅛', '...:)!!', '/!', '$$,', '😒🌚', '$!!', '!!!', 'හවුල', '^', '.&;', '!"', '\\:)/', ':::::)', ':))))))))))))))))))))))))))', "''''", '🍹🍸🍻', '\\', ';__;', "??'", '(!:)).', '♡♥♡', '-.:-)', "'...", '/', '♡♥♡♥', '...!!', '_:', '---:', '🍸╲──╱🍤╭╮╭╮┓┓╭╮╮┳╭🍸╲──╱🍤', '.."', '!?!', ':*', ':))):):)', '❤', "')", '👈💗', '!:)😘', '???', '$,', '😧', '?💕', ':).=', '!!!!', '&;&;', '!”', '::', '--&;', '[', '❤👓💘', '.....:-)', ':)💘', '👱🏽👽🌳', 'ęÿ', '💪', ':):)

In [25]:
# see change in positive samples 
for index in random_indexs:
  # print random tweet
  print('Tweet number:',index,':',cleaned_positive_tweets[index])

Tweet number: 327 : ['Gratefulness', 'Happiness', 'positive', 'Post', 'three', 'things', 'grateful', 'today', 'comments']
Tweet number: 554 : ['thnks', 'progrmr']
Tweet number: 4084 : ['Not', 'sure', 'would', 'good', 'thing', 'bottom', 'daring', 'say', 'Miss', 'gon', 'stubborn', 'mouth', 'soaping', 'positive']
Tweet number: 61 : ['follow', 'amp', 'follow', 'back', 'positive']
Tweet number: 3728 : ['special', 'Acha', 'banda', 'tweets', 'bhi', 'achi', 'krta', 'Handsome', 'bhi', 'Must', 'follow', 'positive']


In [26]:
# see change in negative samples 
for index in random_indexs:
  # print random tweet
  print('Tweet number:',index,':',cleaned_negative_tweets[index])

Tweet number: 327 : ['Such', 'stressful', 'upsetting', 'day', 'yesterday', 'government', 'sucks', 'negative']
Tweet number: 554 : ['ruins', 'know', 'next', 'day', 'wake', 'early', 'morning', 'negative']
Tweet number: 4084 : ['Wish', 'buy', 'Bangtan', 'merch', 'negative']
Tweet number: 61 : ['Well', 'sure', 'work', 'day', 'negative']
Tweet number: 3728 : ['hii', 'vin', 'plss', 'rply', 'tweet', 'negative']


## pre-process data 

In [27]:
# make stem and lemmatize words (positive data)
cleaned_positive_tweets=wordCleaner(cleaned_positive_tweets)
# make stem and lemmatize words (positive data)
cleaned_negative_tweets=wordCleaner(cleaned_negative_tweets)

In [16]:
# see change in positive samples 
for index in random_indexs:
  # print random tweet
  print('Tweet number:',index,':',cleaned_positive_tweets[index])

Tweet number: 4427 : ['overwhelm', 'groupmat', 'thank', 'group', 'get', 'recit', 'posit']
Tweet number: 1657 : ['hell', 'yes', 'will', 'donington', 'park', 'hop', 'see', 'team', 'season', 'our', 'posit']
Tweet number: 4433 : ['mont', 'doa', 'header', 'posit']
Tweet number: 1503 : ['check', 'place', 'avail', 'colour', 'aggreg', 'work', 'tend', 'fade']
Tweet number: 4968 : ['thank', 'mention', 'happi', 'friday', 'posit']


In [28]:
# see change in negative samples 
for index in random_indexs:
  # print random tweet
  print('Tweet number:',index,':',cleaned_negative_tweets[index])

Tweet number: 327 : ['such', 'stress', 'upset', 'day', 'yesterday', 'govern', 'suck', 'negat']
Tweet number: 554 : ['ruin', 'know', 'next', 'day', 'wake', 'earli', 'morn', 'negat']
Tweet number: 4084 : ['wish', 'buy', 'bangtan', 'merch', 'negat']
Tweet number: 61 : ['well', 'sure', 'work', 'day', 'negat']
Tweet number: 3728 : ['hii', 'vin', 'plss', 'rpli', 'tweet', 'negat']


In [40]:
poistiveGenerator=getGenerator(cleaned_positive_tweets)
print(f'type of postive_word_gen_ -> {poistiveGenerator}:',type(poistiveGenerator))
# make frequency dict 
from nltk import FreqDist
# make postive words frequency dictionary 
positive_words_frequency=FreqDist(poistiveGenerator)
# see -> frequency dict 
print(positive_words_frequency)
print('Frequency of top 10 most common elements [positive]:\n',positive_words_frequency.most_common(10),sep='')
type(positive_words_frequency) # to see resutl -> postive_freq_dict_ -> return dict 

type of postive_word_gen_ -> <generator object getGenerator at 0x7f66c63564d0>: <class 'generator'>
<FreqDist with 5227 samples and 29482 outcomes>
Frequency of top 10 most common elements [positive]:
[('posit', 4509), ('thank', 646), ('follow', 443), ('love', 399), ('good', 288), ('get', 273), ('day', 248), ('like', 233), ('happi', 208), ('see', 196)]


nltk.probability.FreqDist

In [41]:
negativeGenerator=getGenerator(cleaned_negative_tweets)
print(f'type of postive_word_gen_ -> {negativeGenerator}:',type(negativeGenerator))
# make frequency dict 
from nltk import FreqDist

# make postive words frequency dictionary 
negative_words_frequency=FreqDist(negativeGenerator)
# see -> frequency dict 
print(negative_words_frequency)
print('Frequency of top 10 most common elements [negative]:\n',negative_words_frequency.most_common(10),sep='')
type(negative_words_frequency) # to see resutl -> postive_freq_dict_ -> return dict 

type of postive_word_gen_ -> <generator object getGenerator at 0x7f66be3744d0>: <class 'generator'>
<FreqDist with 5349 samples and 28399 outcomes>
Frequency of top 10 most common elements [negative]:
[('negat', 4636), ('get', 330), ('miss', 304), ('pleas', 275), ('follow', 263), ('like', 223), ('time', 166), ('feel', 158), ('love', 151), ('day', 150)]


nltk.probability.FreqDist

In [31]:
# make dataset for model (modeled data)
poistive_words4Data=getModeledTokens(cleaned_positive_tweets)
negative_words4Data=getModeledTokens(cleaned_negative_tweets)
# see (first sample)
print('Sample: ',next(getModeledTokens(cleaned_positive_tweets)))

# make dataset - 
positive_dataset=[(tweet_dict,'Positive') for tweet_dict in poistive_words4Data]
negative_dataset=[(tweet_dict,'Negative') for tweet_dict in negative_words4Data]
# merge dataset 
dataset=positive_dataset+negative_dataset
print('Size of dataset:',len(dataset)) 
# split data set -> 70:30 
from sklearn.model_selection import train_test_split
# split data
train_set,test_set=train_test_split(dataset,test_size=0.30)
# see size 
len(train_set),len(test_set) 

Sample:  {'top': True, 'engag': True, 'member': True, 'communiti': True, 'week': True, 'posit': True}
Size of dataset: 10000


(7000, 3000)

In [32]:
# see first sample (final data)
[(tweet_dict,'Positive') for tweet_dict in getModeledTokens(cleaned_positive_tweets)][0]

({'communiti': True,
  'engag': True,
  'member': True,
  'posit': True,
  'top': True,
  'week': True},
 'Positive')

## training and testing model - By using NaiveBayesClassifier



In [33]:
# import module 
from nltk import classify,NaiveBayesClassifier
# train maodel 
classifier_=NaiveBayesClassifier.train(train_set)
# test model 
print('Accuracy score:',round(classify.accuracy(classifier_,test_set)*100,3))

Accuracy score: 95.6


In [34]:
# classifier 10 most informative features
classifier_.show_most_informative_features(10)

Most Informative Features
                   negat = True           Negati : Positi =   2116.7 : 1.0
                   posit = True           Positi : Negati =   1990.9 : 1.0
                     bam = True           Positi : Negati =     21.9 : 1.0
                     sad = True           Negati : Positi =     20.3 : 1.0
                     too = True           Negati : Positi =     16.5 : 1.0
                 appreci = True           Positi : Negati =     15.4 : 1.0
                     via = True           Positi : Negati =     15.4 : 1.0
              bestfriend = True           Positi : Negati =     15.1 : 1.0
                  awesom = True           Positi : Negati =     13.6 : 1.0
                 definit = True           Positi : Negati =     12.8 : 1.0


## testing on hindi input

In [35]:
def conversion(get_inp):
  from googletrans import Translator
  translator = Translator()
  get_inp = translator.translate(text = get_inp,dest = 'en',src = 'hi')
  get_inp = get_inp.text
  return [get_inp]

In [36]:
# sample postive sentence 
inp=input("Enter text in hindi")
custom_tweet=conversion(inp)
cleaned_custom_tweet=removeStopWords(specialCharaterRemover(tagRemover(custom_tweet),positive_similes,
                                                            to_prevent_positive,positive_similes_code))
# preprocessing - 
custom_tokens=wordCleaner(cleaned_custom_tweet)

# print results ## Note classifier_ takes only single sample at a time (not like sklearn classifiers) 
print(custom_tweet,' <- type:',classifier_.classify([dict([token,True] for token in sample) for sample in custom_tokens][0]))

Enter text in hindiभारत सुंदर है
['India is beautiful']  <- type: Positive
